# Weather Analysis

In [15]:
# Dependencies
import sys
import numpy as np

In [16]:
# install citypy
sys.path.append('./lib/python3.8/site-packages')

In [17]:
from citipy import citipy

# test the citipy api
city = citipy.nearest_city(36.81964369493615, 11.053632348043188)
print(f"City Name: {city.city_name}, Country Code: {city.country_code}")

City Name: manzil tamim, Country Code: tn


## API Calls:

In [18]:
# Randomly select at least 500 unique (non-repeat) cities based on latitude and longitude.

# Define min and max longitudes and latitudes. 
# Google says Latitudes range from -90 to 90, and longitudes range from -180 to 180
minlat = -90
maxlat = 90

minlon = -180
maxlon = 180
random_cities = {"city_name":"", "country_code":""}
i = 0

# loop through latitude and longitude range to pick nearest city and append to dictionary

for lat in np.arange(minlat, maxlat, 7.2):
    for lon in np.arange(minlon, maxlon, 6.4):
        city = citipy.nearest_city(lat, lon)
        random_cities.city_name = city.city_name
        random_cities.country_code = city.country_code
        
# save it to a dataframe for quick check
pd.DataFrame(random_cities, index=[0]).T.head()


AttributeError: 'dict' object has no attribute 'city_name'

In [ ]:
# Perform a weather check on each of the cities using a series of successive API calls.

In [ ]:
# Include a print log of each city as it's being processed with the city number and city name

## Scatter Plots:

### Scatter plot for Temperature (F) vs. Latitude

In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Explanation

In [ ]:
# scatter plot for Humidity (%) vs. Latitude


In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Explanation

In [ ]:
# scatter plot for Cloudiness (%) vs. Latitude


In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Explanation

In [ ]:
# scatter plot for Wind Speed (mph) vs. Latitude


In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Explanation

## Linear Regressions:

In [ ]:
# Northern Hemisphere - Temperature (F) vs. Latitude
# Southern Hemisphere - Temperature (F) vs. Latitude

In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Analysis

In [ ]:
# Northern Hemisphere - Humidity (%) vs. Latitude
# Southern Hemisphere - Humidity (%) vs. Latitude

In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Analysis

In [ ]:
# Northern Hemisphere - Cloudiness (%) vs. Latitude
# Southern Hemisphere - Cloudiness (%) vs. Latitude

In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Analysis

In [ ]:
# Northern Hemisphere - Wind Speed (mph) vs. Latitude
# Southern Hemisphere - Wind Speed (mph) vs. Latitude

In [ ]:
## Save a CSV of all retrieved data and a PNG image for each scatter plot

In [ ]:
## Analysis